**This is a sample of how to use Image_Caption class**

## Image Captioning based Model
- acquired image caption from the pre-trained image captioning model (https://github.com/ruotianluo/ImageCaptioning.pytorch)
- caculate the wmd between the each pair of image caption and article title
- sort the potential matched image list by the wmd

In [13]:
from scipy import spatial
import gensim.downloader as api

In [14]:
model = api.load("word2vec-google-news-300")

In [58]:
def get_caption (caption_file):
    print(caption_file)
    articles_names=open(caption_file, 'r', encoding="utf-8")
    lines = [line.strip() for line in articles_names]
    result_dict={}
    for i in range(len(lines)):
        orig_line=lines[i]
        segs = orig_line.split("\t")
        if segs[0] not in result_dict:
            result_dict[segs[0]]=segs[1]
    return result_dict

In [59]:
def get_ar_id_title (article_file, title_eng_idx):
    print(article_file)
    articles_names=open(article_file, 'r', encoding="utf-8")
    next(articles_names)
    lines = [line.strip() for line in articles_names]
    result_dict={}
    for i in range(len(lines)):
        orig_line=lines[i]
        segs = orig_line.split("\t")
        if len(segs)>=3 and segs[0] not in result_dict:
            result_dict[segs[0]]=segs[title_eng_idx]
    return result_dict

In [60]:
def cal_sim(id_title, caption_dict):
    sim_result = {}
    cnt = 0
    for ar_id, title in id_title.items():
        caption_sim=[]
        for img_id, caption in caption_dict.items():
            sim = model.wmdistance(title, caption)
            caption_sim.append((img_id, sim))
        cnt+=1
        sim_result[ar_id]=caption_sim
        #print(str(datetime.now()))
        if cnt%100 == 0:
            print("complete ", cnt, " article")
    return sim_result

In [61]:
def write_wmd_sim(wmd_sim_file, sim_result):
    f = open(wmd_sim_file, "a")
    for key, v in sim_result.items():
        for item in v:
            result=key+"\t"+os.path.basename(item[0])+"\t"+str(item[1])+"\n"
            f.write(result)
    f.close()

In [74]:
from datetime import datetime
def img_cap_similarity(caption_file, title_file, wmd_sim_file, title_eng_idx):
    caption_dict=get_caption (caption_file)
    ar_id_title=get_ar_id_title (title_file, title_eng_idx)
    sim_result=cal_sim(ar_id_title, caption_dict)
    write_wmd_sim(wmd_sim_file, sim_result)
    lines_caption_sim = []
    with open(wmd_sim_file) as f:
        lines_caption_sim = f.readlines()
    ar_cap_sim_dic={}
    for line in lines_caption_sim:
        segs=line.strip().split("\t")
        if segs[0] not in ar_cap_sim_dic:
            ar_cap_sim_dic[segs[0]]={}
        ar_cap_sim_dic[segs[0]][segs[1]]= -float(segs[2])
    sort_final_result=sort_dict(ar_cap_sim_dic)
    return sort_final_result

In [75]:
def sort_dict(sim_result):
    result={}
    for k, v in sim_result.items():
        sort_v=dict(sorted(v.items(), key=lambda item: item[1]))
        result[k]=sort_v
    return result

In [76]:
def eval_cap_sim (sort_final_result):
    count=0
    for key, value in sort_final_result.items():
        first_tuple_elements=[]
        for a_tuple in value:
            first_tuple_elements.append(a_tuple)
        if key in first_tuple_elements[0:100]:
            count+=1
    return count/len(sort_final_result)

In [ ]:
train_result = img_cap_similarity(r"processed_data\data\train_image_caption_result.txt", \
                                  r"processed_data\data\train_title_eng.tsv",\
                                  r"result/tr_caption_sim_wmd.tsv", 3)

In [ ]:
eval_result = img_cap_similarity(r"processed_data\data\eval_image_caption_result.txt", \
                                 "processed_data\data\eval_title_eng.tsv",\
                                "result/eval_caption_sim_wmd.tsv", 3)

In [ ]:
eval_cap_sim (eval_result)

In [ ]:
test_result = img_cap_similarity(r"processed_data\data\test_image_caption_result.txt", \
                                 "processed_data\data\test_title_eng.tsv",\
                                "result/test_caption_sim_wmd.tsv", 2)